In [56]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
%autoreload 
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML
import utilities as ut

Start h2o client

In [58]:
h2o.init(max_mem_size='16G')

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 42 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_henge_64i7ig
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,15.69 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


### Load FE data


In [59]:
X_train = pd.read_csv('X_train_selected.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test_selected.csv')

In [62]:
# set date_foercast as index
X_train.set_index('date_forecast', inplace=True)
y_train.set_index('date_forecast', inplace=True)
X_test.set_index('date_forecast', inplace=True)

In [63]:
X_train_h2o = h2o.H2OFrame(X_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [64]:
db = pd.concat([X_train, y_train], axis=1)
# remove duplicate columns
db = db.loc[:,~db.columns.duplicated()]

In [65]:
db_h20 = h2o.H2OFrame(db)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [66]:
y = "pv_measurement"

x = list(db_h20.columns)
x.remove(y)

## Run AutoML

In [13]:
%%time
aml = H2OAutoML(max_models=20, max_runtime_secs=12000)
aml.train(x=x, y=y, training_frame=db_h20)

AutoML progress: |


18:23:48.875: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
CPU times: total: 8.22 s
Wall time: 44min 40s


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/20
# GBM base models (used / total),6/10
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),1/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [14]:
lb = aml.leaderboard
lb.head(rows=15)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_2_20231011_182348,129.354,16732.5,31.4077,nan,16732.5
StackedEnsemble_BestOfFamily_1_AutoML_2_20231011_182348,129.735,16831.1,31.7319,nan,16831.1
GBM_4_AutoML_2_20231011_182348,129.953,16887.7,31.0064,nan,16887.7
GBM_grid_1_AutoML_2_20231011_182348_model_2,134.017,17960.6,33.3956,nan,17960.6
GBM_3_AutoML_2_20231011_182348,136.416,18609.3,39.4241,nan,18609.3
GBM_1_AutoML_2_20231011_182348,139.747,19529.2,44.6807,nan,19529.2
GBM_grid_1_AutoML_2_20231011_182348_model_1,141.677,20072.3,40.4572,nan,20072.3
GBM_grid_1_AutoML_2_20231011_182348_model_3,142.306,20251,42.6995,nan,20251
GBM_5_AutoML_2_20231011_182348,143.851,20693.1,49.4254,nan,20693.1
GBM_2_AutoML_2_20231011_182348,143.949,20721.2,47.5951,nan,20721.2


In [17]:
aml.leader #Best model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/20
# GBM base models (used / total),6/10
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),1/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [23]:
model = h2o.get_model('StackedEnsemble_AllModels_1_AutoML_2_20231011_182348')

In [44]:
pred = model.predict(X_test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [45]:
pred.shape

(84798, 1)

In [52]:
X_test

,date_forecast,elevationm,sun_elevationd,direct_rad_1hJ,sun_azimuthd,effective_cloud_coverp,msl_pressurehPa,t_1000hPaK,day_of_year,wind_speed_u_10mms,ceiling_height_aglm,visibilitym,relative_humidity_1000hPap,year,day_of_month,day_of_week,fresh_snow_24hcm,fresh_snow_6hcm,fresh_snow_1hcm,pv_measurement
0,2019-09-04 08:00:00,7.0,23.775,914199.4,124.573,0.3,1009.3,283.0,247,-0.3,2103.1,45762.0,66.1,2019,4,2,0.0,0.0,0.0,570.1125
1,2019-09-04 08:00:00,7.0,23.775,914199.4,124.573,0.3,1009.3,283.0,247,-0.3,2103.1,45762.0,66.1,2019,4,2,0.0,0.0,0.0,570.1125
2,2019-09-04 08:00:00,7.0,23.775,914199.4,124.573,0.3,1009.3,283.0,247,-0.3,2103.1,45762.0,66.1,2019,4,2,0.0,0.0,0.0,570.1125
3,2019-09-04 09:00:00,6.0,28.732,1239638.2,140.277,43.0,1008.5,284.2,247,-1.3,2478.4,51142.4,58.8,2019,4,2,0.0,0.0,0.0,3400.7600
4,2019-09-04 09:00:00,6.0,28.732,1239638.2,140.277,43.0,1008.5,284.2,247,-1.3,2478.4,51142.4,58.8,2019,4,2,0.0,0.0,0.0,3400.7600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84793,2023-04-30 22:00:00,6.0,-10.630,0.0,341.851,95.4,1014.5,274.4,120,3.4,1532.9,16674.9,78.4,2023,30,6,0.0,0.0,0.0,0.0000
84794,2023-04-30 22:00:00,6.0,-10.630,0.0,341.851,95.4,1014.5,274.4,120,3.4,1532.9,16674.9,78.4,2023,30,6,0.0,0.0,0.0,0.0000
84795,2023-04-30 23:00:00,24.0,-11.923,0.0,356.630,100.0,1014.4,274.1,120,1.9,897.2,9890.9,80.4,2023,30,6,0.0,0.0,0.0,-0.0000
84796,2023-04-30 23:00:00,24.0,-11.923,0.0,356.630,100.0,1014.4,274.1,120,1.9,897.2,9890.9,80.4,2023,30,6,0.0,0.0,0.0,-0.0000
